# Подключение гугл диска

In [1]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive/3_year/WSI

/content/gdrive/MyDrive/3_year/WSI


# Установки и импорты

In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached deep_translator-1.11.0-py3-none-any.whl (37 kB)
  Using cached fasttext_wheel-0.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.4 MB)
  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)
  Using cached sklearn-0.0.post5.tar.gz (3.7 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 45.3 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0.post5-py3-none-any.whl size=2950 sha256=08315d507a2aaa6030c50e8006ac3546138f8f309d104db7e10caf141122b69e
  Stored in directory: /root/.cache/pip/wheels/38/1f/8d/4f812c590e074c1e928f5cec67bf5053b71f38e2648739403a
Successfully built sklearn


In [4]:
import fasttext
from pathlib import Path

In [5]:
import sys
sys.path.append('/content/gdrive/MyDrive/3_year/WSI/lexsubgen/')

In [6]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/gdrive/MyDrive/3_year/WSI/lexsubgen/']

In [7]:
import importlib

# Работа с датасетом

In [8]:
dataset_path = Path('./semeval-2010/dataset_wsi_2010.csv').resolve()
dataset_path

PosixPath('/content/gdrive/MyDrive/3_year/WSI/semeval-2010/dataset_wsi_2010.csv')

In [9]:
import pandas as pd

In [10]:
dataset_2010 = pd.read_csv(dataset_path)
dataset_2010.head()

,context_id,group_by,target_lemma,pos_tag,sentence,target_id
0,threat.n.1,threat.n,threat,n,"['However', ',', 'history', 'has', 'proven', '...",58
1,threat.n.2,threat.n,threat,n,"['This', 'report', 'predicted', 'that', 'by', ...",75
2,threat.n.3,threat.n,threat,n,"['A', 'department', 'spokesman', 'said', 'the'...",42
3,threat.n.4,threat.n,threat,n,"['``', 'But', 'the', 'question', 'is', 'whethe...",27
4,threat.n.5,threat.n,threat,n,"['According', 'to', 'the', 'transcript', ',', ...",38


# Обработка контекста

In [12]:
from lexsubgen.fasttext_substitutes import FasttextSubstituteGenerator

In [ ]:
importlib.reload(sys.modules['lexsubgen.fasttext_substitutes'])

### Генерация эмбеддингов контекста

In [ ]:
context_emb_p = Path('./semeval-2010/pos_context_embs.json').resolve()
context_emb_p

In [ ]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
# !gunzip cc.en.300.bin.gz

In [ ]:
eng_model = fasttext.load_model('cc.en.300.bin')

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
eng_substitutor = FasttextSubstituteGenerator(dataset_2010, eng_model, subst_lang='en', context_retriever='pos_excluding')

In [ ]:
eng_substitutor.get_context_embeddings(eng_model)

In [ ]:
eng_substitutor._write_context_embs(eng_substitutor.context_embs, eng_substitutor.target2context2idx, save_path=context_emb_p)

# Генерация подстановок

## Английский

In [ ]:
# скачиваем языковую модель eng_model

# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
# !gunzip cc.en.300.bin.gz
# eng_model = fasttext.load_model('cc.en.300.bin')

In [ ]:
context_emb_p = Path('./semeval-2010/context_embs.json').resolve()
context_emb_p

PosixPath('/content/gdrive/MyDrive/3_year/WSI/semeval-2010/semeval_2010_context_embs.json')

In [ ]:
en_matrix_path = str(Path('./word_alignment/alignment_matrices/en.txt').resolve())
en_matrix_path

In [ ]:
eng_substitutor = FasttextSubstituteGenerator(dataset_2010, eng_model, subst_lang='en', path_to_align_matrix=en_matrix_path, context_retriever='dummy')

In [ ]:
eng_substitutor._read_context_embs(context_emb_p)

In [ ]:
eng_substitutes = eng_substitutor.generate()

In [ ]:
subst_folder = Path('./semeval-2010/substitutes').resolve()
subst_folder
filename_dummy, filename_pos = 'eng_substitutes_dummy.json', 'eng_substitutes_pos.json'

In [ ]:
eng_substitutor._save_substitutes(
    eng_substitutes, 
    lang='en',
    save_dir=subst_folder,
    filename=filename_dummy
)

In [ ]:
context_emb_p = Path('./semeval-2010/pos_context_embs.json').resolve()
context_emb_p

In [ ]:
eng_substitutor = FasttextSubstituteGenerator(dataset_2010, eng_model, subst_lang='en', path_to_align_matrix=en_matrix_path, context_retriever='pos_excluding')
eng_substitutor._read_context_embs(context_emb_p)
eng_substitutes = eng_substitutor.generate()

In [ ]:
eng_substitutor._save_substitutes(
    eng_substitutes, 
    lang='en',
    save_dir=subst_folder,
    filename=filename_pos
)

In [ ]:
del eng_model
del eng_substitutes

In [ ]:
open('./fasttext_models/cc.en.300.bin', 'w').close()
%rm ./fasttext_models/cc.en.300.bin

# Генерация подстановок на разных языках

In [13]:
subst_folder = Path('./semeval-2010/substitutes').resolve()
subst_folder

PosixPath('/content/gdrive/MyDrive/3_year/WSI/semeval-2010/substitutes')

In [14]:
transl_folder = Path('./semeval-2010/target_translations').resolve()
transl_folder

PosixPath('/content/gdrive/MyDrive/3_year/WSI/semeval-2010/target_translations')

In [20]:
dummy_context_emb_p = Path('./semeval-2010/context_embs.json').resolve()
pos_context_emb_p = Path('./semeval-2010/pos_context_embs.json').resolve()
dummy_context_emb_p, pos_context_emb_p

(PosixPath('/content/gdrive/MyDrive/3_year/WSI/semeval-2010/context_embs.json'),
 PosixPath('/content/gdrive/MyDrive/3_year/WSI/semeval-2010/pos_context_embs.json'))

In [ ]:
targets = list(dataset_2010['group_by'].unique())
targets

In [66]:
def fix_translations(transl_dict, lang, change_transl=None):
    if lang == 'fr':
        "Убираем возвратную частицу se"
        transl_dict = {w: t.replace('se ', '') for w, t in transl_dict.items()}
    if change_transl:
        for w, tr in change_transl.items():
            print(w, tr)
            transl_dict[w] = tr
    return transl_dict

## Французский

In [ ]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.bin.gz -P ./fasttext_models/
# !gunzip cc.fr.300.bin.gz
fr_model = fasttext.load_model('cc.fr.300.bin')

In [ ]:
fr_matrix_path = str(Path('./word_alignment/alignment_matrices/fr.txt').resolve())
fr_matrix_path

In [ ]:
fr_substitutor = FasttextSubstituteGenerator(dataset_2010, fr_model, subst_lang='fr', path_to_align_matrix=fr_matrix_path, context_retriever='dummy')
fr_substitutor._read_context_embs(dummy_context_emb_p)

In [ ]:
targets = list(fr_substitutor.context_embs.keys())
translations = fr_substitutor._translate_targets(targets, 'en', 'fr')

In [ ]:
fixed_fr_targets = fix_translations(translations, 'fr', {'screen.n': 'écran', 'house.n': 'maison'})
fr_substitutor._save_translations(fixed_fr_targets, 'fr', transl_folder)

In [ ]:
fr_substitutes = fr_substitutor.generate(translated_targets=fixed_fr_targets)

In [ ]:
fr_substitutor._save_substitutes(
    fr_substitutes, 
    lang='fr',
    save_dir=subst_folder,
    filename='fr_substitutes_dummy.json'
)

In [ ]:
fr_substitutor = FasttextSubstituteGenerator(dataset_2010, fr_model, subst_lang='fr', path_to_align_matrix=fr_matrix_path, context_retriever='pos_excluding')
fr_substitutor._read_context_embs(pos_context_emb_p)
fr_substitutes = fr_substitutor.generate(translated_targets=fixed_fr_targets)
fr_substitutor._save_substitutes(
    fr_substitutes, 
    lang='fr',
    save_dir=subst_folder,
    filename='fr_substitutes_pos.json'
)


## Испанский

In [15]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.bin.gz -P ./fasttext_models/
!gunzip ./fasttext_models/cc.es.300.bin.gz

--2023-05-21 16:36:09--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.7.50, 13.35.7.128, 13.35.7.38, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.7.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4500107671 (4.2G) [application/octet-stream]
Saving to: ‘./fasttext_models/cc.es.300.bin.gz’

cc.es.300.bin.gz    100%[===================>]   4.19G  37.8MB/s    in 2m 10s  

2023-05-21 16:38:19 (33.1 MB/s) - ‘./fasttext_models/cc.es.300.bin.gz’ saved [4500107671/4500107671]



In [16]:
es_model = fasttext.load_model('./fasttext_models/cc.es.300.bin')

In [17]:
es_matrix_path = str(Path('./word_alignment/alignment_matrices/es.txt').resolve())
es_matrix_path

'/content/gdrive/MyDrive/3_year/WSI/word_alignment/alignment_matrices/es.txt'

In [18]:
es_substitutor = FasttextSubstituteGenerator(dataset_2010, es_model, subst_lang='es', path_to_align_matrix=es_matrix_path, context_retriever='dummy')

In [21]:
es_substitutor._read_context_embs(dummy_context_emb_p)

In [ ]:
translations = es_substitutor._translate_targets(targets, 'en', 'es')
translations

In [70]:
fix_es_translations = fix_translations(translations, 'es', {'cell.n': 'célula'})

cell.n célula


In [72]:
es_substitutor._save_translations(fix_es_translations, 'es', transl_folder)

PosixPath('/content/gdrive/MyDrive/3_year/WSI/semeval-2010/target_translations/es_target.json')

In [73]:
es_substitutes = es_substitutor.generate(translated_targets=fix_es_translations)

100%|██████████| 100/100 [04:29<00:00,  2.69s/it]


In [74]:
es_substitutor._save_substitutes(
    es_substitutes, 
    lang='es',
    save_dir=subst_folder,
    filename='es_substitutes_dummy.json'
)

In [75]:
es_substitutor = FasttextSubstituteGenerator(dataset_2010, es_model, subst_lang='es', path_to_align_matrix=es_matrix_path, context_retriever='pos_excluding')
es_substitutor._read_context_embs(pos_context_emb_p)
es_substitutes = es_substitutor.generate(translated_targets=fix_es_translations)
es_substitutor._save_substitutes(
    es_substitutes, 
    lang='es',
    save_dir=subst_folder,
    filename='es_substitutes_pos.json'
)

100%|██████████| 100/100 [04:12<00:00,  2.53s/it]


In [76]:
# удаляем переменные и файл 

del es_model
del es_substitutes
open('./fasttext_models/cc.es.300.bin', 'w').close()
%rm ./fasttext_models/cc.es.300.bin

In [77]:
del es_substitutor

In [78]:
import gc
gc.collect()

38386

## Немецкий

In [79]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.bin.gz -P ./fasttext_models/
!gunzip ./fasttext_models/cc.de.300.bin.gz
de_model = fasttext.load_model('./fasttext_models/cc.de.300.bin')

--2023-05-21 17:54:03--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.7.128, 13.35.7.82, 13.35.7.38, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.7.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4496980947 (4.2G) [application/octet-stream]
Saving to: ‘./fasttext_models/cc.de.300.bin.gz’

cc.de.300.bin.gz    100%[===================>]   4.19G  36.2MB/s    in 2m 2s   

2023-05-21 17:56:06 (35.1 MB/s) - ‘./fasttext_models/cc.de.300.bin.gz’ saved [4496980947/4496980947]



In [80]:
de_matrix_path = str(Path('./word_alignment/alignment_matrices/de.txt').resolve())
de_matrix_path

'/content/gdrive/MyDrive/3_year/WSI/word_alignment/alignment_matrices/de.txt'

In [81]:
de_substitutor = FasttextSubstituteGenerator(dataset_2010, de_model, subst_lang='de', path_to_align_matrix=de_matrix_path, context_retriever='dummy')
de_substitutor._read_context_embs(dummy_context_emb_p)

In [ ]:
translations = de_substitutor._translate_targets(targets, 'en', 'de')
translations

In [84]:
for w, tr in translations.items():
    if de_model.get_word_id(tr) == -1:
        print(w, tr)

conclude.v daraus schließen


In [92]:
fix_de_translations = fix_translations(translations, 'de', {'conclude.v': 'schließen'})

conclude.v schließen


In [93]:
de_substitutor._save_translations(fix_de_translations, 'de', transl_folder)

PosixPath('/content/gdrive/MyDrive/3_year/WSI/semeval-2010/target_translations/de_target.json')

In [94]:
de_substitutes = de_substitutor.generate(translated_targets=fix_de_translations)

100%|██████████| 100/100 [05:59<00:00,  3.59s/it]


In [97]:
de_substitutes.keys()

dict_keys(['access.n', 'accommodate.v', 'accounting.n', 'address.n', 'air.n', 'analyze.v', 'appeal.v', 'apply.v', 'assemble.v', 'assert.v', 'body.n', 'bow.v', 'camp.n', 'campaign.n', 'cell.n', 'challenge.n', 'cheat.v', 'chip.n', 'class.n', 'commission.n', 'commit.v', 'community.n', 'conclude.v', 'cultivate.v', 'dealer.n', 'defend.v', 'deny.v', 'deploy.v', 'display.n', 'divide.v', 'edge.n', 'entry.n', 'expose.v', 'failure.n', 'field.n', 'figure.v', 'flight.n', 'foundation.n', 'frame.v', 'function.n', 'gap.n', 'gas.n', 'guarantee.n', 'happen.v', 'haunt.v', 'house.n', 'idea.n', 'innovation.n', 'insist.v', 'introduce.v', 'lay.v', 'legislation.n', 'level.v', 'lie.v', 'margin.n', 'mark.n', 'market.n', 'mind.n', 'moment.n', 'mount.v', 'movement.n', 'note.n', 'observe.v', 'office.n', 'officer.n', 'operate.v', 'origin.n', 'owe.v', 'park.n', 'pour.v', 'presume.v', 'promotion.n', 'pursue.v', 'question.v', 'rally.n', 'reap.v', 'regain.v', 'relax.v', 'reputation.n', 'reveal.v', 'road.n', 'root.v', 

In [ ]:
for t in de_substitutes:
    print(t)
    print(de_substitutes[t][t+'.1'])

In [100]:
de_substitutor._save_substitutes(
    de_substitutes, 
    lang='de',
    save_dir=subst_folder,
    filename='de_substitutes_dummy.json'
)

In [101]:
de_substitutor = FasttextSubstituteGenerator(dataset_2010, de_model, subst_lang='de', path_to_align_matrix=de_matrix_path, context_retriever='pos_excluding')
de_substitutor._read_context_embs(pos_context_emb_p)
de_substitutes = de_substitutor.generate(translated_targets=fix_de_translations)
de_substitutor._save_substitutes(
    de_substitutes, 
    lang='de',
    save_dir=subst_folder,
    filename='de_substitutes_pos.json'
)

100%|██████████| 100/100 [05:41<00:00,  3.41s/it]


In [104]:
# удаляем переменные и файл 

# del de_model
# del de_substitutes
del de_substitutor
open('./fasttext_models/cc.de.300.bin', 'w').close()
%rm ./fasttext_models/cc.de.300.bin

In [105]:
gc.collect()

28099

## Русский

In [106]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz -P ./fasttext_models/
!gunzip ./fasttext_models/cc.ru.300.bin.gz
ru_model = fasttext.load_model('./fasttext_models/cc.ru.300.bin')

--2023-05-21 18:23:18--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.7.50, 13.35.7.128, 13.35.7.82, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.7.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4496459151 (4.2G) [application/octet-stream]
Saving to: ‘./fasttext_models/cc.ru.300.bin.gz’

cc.ru.300.bin.gz    100%[===================>]   4.19G  34.5MB/s    in 2m 8s   

2023-05-21 18:25:27 (33.6 MB/s) - ‘./fasttext_models/cc.ru.300.bin.gz’ saved [4496459151/4496459151]



In [107]:
ru_matrix_path = str(Path('./word_alignment/alignment_matrices/ru.txt').resolve())
ru_matrix_path

'/content/gdrive/MyDrive/3_year/WSI/word_alignment/alignment_matrices/ru.txt'

In [108]:
ru_substitutor = FasttextSubstituteGenerator(dataset_2010, ru_model, subst_lang='ru', path_to_align_matrix=ru_matrix_path, context_retriever='dummy')
ru_substitutor._read_context_embs(dummy_context_emb_p)

In [ ]:
translations = ru_substitutor._translate_targets(targets, 'en', 'ru')
translations

In [110]:
for w, tr in translations.items():
    if ru_model.get_word_id(tr) == -1:
        print(w, tr)

accounting.n бухгалтерский учет
owe.v быть должным


In [117]:
ru_model.get_nearest_neighbors('должный')

[(0.7749164700508118, 'надлежащий'),
 (0.7416205406188965, 'Должный'),
 (0.6413440108299255, 'достаточный'),
 (0.6368164420127869, 'подобающий'),
 (0.6147304773330688, 'должную'),
 (0.5891010761260986, 'высочайший'),
 (0.5881335139274597, 'требуемый'),
 (0.5876595973968506, 'необходимый'),
 (0.5826200842857361, 'пристойный'),
 (0.5729640126228333, 'безупречный')]

In [118]:
fix_ru_translations = fix_translations(translations, 'ru', {'accounting.n': 'учёт', 'owe.v': 'должный'})

accounting.n учёт
owe.v должный


In [119]:
ru_substitutor._save_translations(fix_ru_translations, 'ru', transl_folder)

PosixPath('/content/gdrive/MyDrive/3_year/WSI/semeval-2010/target_translations/ru_target.json')

In [120]:
ru_substitutes = ru_substitutor.generate(translated_targets=fix_ru_translations)

100%|██████████| 100/100 [04:50<00:00,  2.91s/it]


In [121]:
for t in ru_substitutes:
    print(ru_substitutes[t][t+'.1'])

преградить пропуск проникнуть допуск несанкционированный проход просочиться проникновение запаролить открыть позволен фтп подступ демо-доступ паролям въезд доступ неавторизованный оперативный запароленный перекрыть блокировать привилегированный логинам выход онлайн-контенту публичный парольный удаленный администраторский root-доступ защищенный аккаунту контакт закрыт заблокированному непосредственный файлообмен закрытому хранилищу файлам равный люк онлайн-доступ интернет-ресурсам одновременный синхронизированный немедленный приватный преграждающий веб-ресурсам загружаемому переход потоковому просмотр вай-фаю пользовательский дополнительный подсоединяться расширенный законнектиться веб-интерфейс неподконтрольный секретный серверам контенту шифрованный конфиденциальной разрешен онлайн-услугам транзит круглосуточный интернет свободный премиум-доступ бесконтрольный приватному перекройте архивам сеансовый мастер-ключ коммутируемый сервисам монопольный высокоскоростному инет преимущественный

In [122]:
ru_substitutor._save_substitutes(
    ru_substitutes, 
    lang='ru',
    save_dir=subst_folder,
    filename='ru_substitutes_dummy.json'
)

In [123]:
ru_substitutor = FasttextSubstituteGenerator(dataset_2010, ru_model, subst_lang='ru', path_to_align_matrix=ru_matrix_path, context_retriever='pos_excluding')
ru_substitutor._read_context_embs(pos_context_emb_p)
ru_substitutes = ru_substitutor.generate(translated_targets=fix_ru_translations)
ru_substitutor._save_substitutes(
    ru_substitutes, 
    lang='ru',
    save_dir=subst_folder,
    filename='ru_substitutes_pos.json'
)

100%|██████████| 100/100 [04:49<00:00,  2.89s/it]


In [124]:
# удаляем переменные и файл 

del ru_model
del ru_substitutes
del ru_substitutor
open('./fasttext_models/cc.ru.300.bin', 'w').close()
%rm ./fasttext_models/cc.ru.300.bin

In [125]:
gc.collect()

27925

# Кластеризация

In [ ]:
importlib.reload(sys.modules['lexsubgen.clusterizer'])

<module 'lexsubgen.clusterizer' from '/content/gdrive/MyDrive/3_year/WSI/lexsubgen/clusterizer.py'>

In [ ]:
from lexsubgen.clusterizer import SubstituteClusterizer

In [ ]:
clust = SubstituteClusterizer(n_clusters='maxsil=range(2, 10)')

In [ ]:
subst_folder = '/content/gdrive/MyDrive/3_year/WSI/semeval-2010/substitutes'
lang_substitutes = [f.absolute() for f in Path(subst_folder).iterdir()]
lang_substitutes

[PosixPath('/content/gdrive/MyDrive/3_year/WSI/semeval-2010/substitutes/fr_substitutes.json'),
 PosixPath('/content/gdrive/MyDrive/3_year/WSI/semeval-2010/substitutes/es_substitutes.json'),
 PosixPath('/content/gdrive/MyDrive/3_year/WSI/semeval-2010/substitutes/de_substitutes.json'),
 PosixPath('/content/gdrive/MyDrive/3_year/WSI/semeval-2010/substitutes/ru_substitutes.json'),
 PosixPath('/content/gdrive/MyDrive/3_year/WSI/semeval-2010/substitutes/eng_substitutes.json')]

In [ ]:
clust_folder = Path('./semeval-2010/clusterization').resolve()
clust_folder

PosixPath('/content/gdrive/MyDrive/3_year/WSI/semeval-2010/clusterization')

In [ ]:
clust_res = clust.cluster_all(lang_substitutes, n_subst=5)
clust._save_clusterization(clust_res, clust_folder)

PosixPath('/content/gdrive/MyDrive/3_year/WSI/semeval-2010/clusterization/clusterization.json')

# Подсчет метрик

In [ ]:
importlib.reload(sys.modules['lexsubgen.evaluation'])

In [ ]:
from lexsubgen.evaluation import WSIEvaluator

In [ ]:
eval = WSIEvaluator(dataset=dataset_2010,
                    clust_res_path='/content/gdrive/MyDrive/3_year/WSI/semeval-2010/clusterization/clusterization.json',
                    semeval_data_path='/content/gdrive/MyDrive/3_year/WSI/semeval-2010')

In [ ]:
eval.compute_metrics()

fscore	precision	recall	vmeasure	homogenity\completness


,fscore,precision,recall,vmeasure,homogenity,completeness,(fs * vm) ** 0.5
threat.n,63.3,91.1,48.6,3.2,10.2,1.9,14.232357
address.n,57.6,98.1,40.7,8.3,41.2,4.6,21.865041
camp.n,55.6,95.1,39.2,6.3,27.5,3.5,18.715769
innovation.n,66.2,92.8,51.5,3.5,10.3,2.1,15.221695
mind.n,38.1,95.5,23.8,3.4,23.2,1.8,11.381564
...,...,...,...,...,...,...,...
sniff.v,41.9,81.2,28.3,9.1,26.4,5.5,19.526648
straighten.v,48.2,92.8,32.6,3.6,18.1,2.0,13.172699
pursue.v,88.2,93.5,83.4,3.7,5.2,2.8,18.064883
defend.v,70.8,93.4,57.0,0.0,0.0,0.0,0.000000
